# Polyhedron Library Comparisons

Let's compare a few different polyhedron libraries and their python and Julia implementations. Specifically, we'd like to see if they all agree about the vertices of a particular polyhedron. 

In [1]:
# Pkg.add("CDDLib")
# Pkg.add("ConvexHull")
# Pkg.add("QHull")
# Pkg.add("PyCall")
# run(`pip install pycddlib`)
# run(`pip install pyhull`)

In [2]:
using Polyhedra
using PyCall
import CDDLib
import ConvexHull
import QHull

Here is the description of our polyhedron, described as:

$$
\{ x \mid A x \leq b \}
$$

In [3]:
A = [
   9.55116190e-01 2.60905924e-01 1.22949243e-01 6.75621594e-02;
   1.99840144e-15 4.26325641e-14 2.46469511e-14 3.19744231e-14;
  -9.59651490e-01 2.05071779e-01  -1.91930298e-01 1.33170253e-02;
   9.59651490e-01  -2.05071779e-01 1.91930298e-01  -1.33170253e-02;
   9.88052600e-01  -5.06184465e-02 1.45490937e-01 4.71367055e-03;
  -9.88052600e-01 5.06184465e-02  -1.45490937e-01  -4.71367055e-03;
  -2.31041437e-01  -9.53299309e-01 5.12455741e-02  -1.87654399e-01;
   2.31041437e-01 9.53299309e-01  -5.12455741e-02 1.87654399e-01;
   9.93765086e-01  -5.05219325e-02 9.93765086e-02  -1.67291698e-03;
  -9.10739437e-18 9.95627124e-01  -1.12758216e-16 9.34164361e-02;
  -1.39101357e-16  -7.18266149e-01 6.94871051e-01  -3.53264860e-02;
  -8.68205735e-17 8.82512712e-01  -1.09046640e-15 4.70288543e-01;
  -9.93765086e-01 5.05219325e-02  -9.93765086e-02 1.67291698e-03;
   9.10739437e-18  -9.95627124e-01 1.12758216e-16  -9.34164361e-02;
   1.39101357e-16 7.18266149e-01  -6.94871051e-01 3.53264860e-02;
   8.68205735e-17  -8.82512712e-01 1.09046640e-15  -4.70288543e-01;
  -1.11022302e-16 0.00000000e+00  -3.33066907e-16  -3.05311332e-16;
  -5.21702259e-01 8.37724120e-01  -1.04340452e-01 1.23118321e-01;
  -9.84302468e-01 9.72873856e-02  -1.47222630e-01 3.05179109e-03;
   9.51781915e-01 2.75237706e-01 1.16680774e-01 6.88548412e-02;
   5.21702259e-01  -8.37724120e-01 1.04340452e-01  -1.23118321e-01;
   9.84302468e-01  -9.72873856e-02 1.47222630e-01  -3.05179109e-03;
  -1.88737914e-15 4.44089210e-16  -4.44089210e-16  -2.77555756e-16;
  -4.43489353e-01 8.75100775e-01  -1.22613792e-01 1.49938937e-01;
   9.83362018e-01  -1.05147557e-01 1.48055100e-01  -4.77709484e-03;
  -7.20891513e-01  -6.77108732e-01  -4.32955241e-02  -1.41296458e-01;
   4.43489353e-01  -8.75100775e-01 1.22613792e-01  -1.49938937e-01;
  -9.83362018e-01 1.05147557e-01  -1.48055100e-01 4.77709484e-03;
   7.36381565e-01  -6.47640205e-01 1.59986136e-01  -1.12733273e-01;
  -9.34440936e-02 9.75368984e-01  -8.82928243e-02 1.79242640e-01;
   4.50927337e-01  -8.67189950e-01 1.38700060e-01  -1.59400192e-01;
   7.31068112e-01  -6.58536853e-01 1.44970655e-01  -1.04173600e-01;
  -7.36381565e-01 6.47640205e-01  -1.59986136e-01 1.12733273e-01;
   9.34440936e-02  -9.75368984e-01 8.82928243e-02  -1.79242640e-01;
  -4.50927337e-01 8.67189950e-01  -1.38700060e-01 1.59400192e-01;
  -7.31068112e-01 6.58536853e-01  -1.44970655e-01 1.04173600e-01;
  ];

b = [ -9.8327944,
  40.,
   9.96723402,
  -9.77691384,
 -10.03184225,
  10.13520694,
   2.58153898,
  -2.42013871,
   9.83910803,
   0.38130563,
   5.58236351,
   1.911436  ,
  10.03619369,
   0.56508566,
   8.31505752,
   3.67646142,
  19.19099116,
   5.46751028,
  10.53487023,
  -9.77601526,
  -4.93470758,
  -9.54211296,
  17.11656705,
   4.58420794,
  -9.53345218,
   7.47289366,
  -4.23783232,
  10.52264718,
   0.99834034,
   1.01444088,
  -4.17636956,
  -6.65146053,
   7.42622548,
  -0.66005808,
   4.78145376,
   7.97833638];

## Python Libraries

Let's set up python functions to retrieve the vertices using cddlib and qhull, provided by the `pycddlib` and `pyhull` packages:

In [4]:
py"""
import numpy as np
import cdd
from pyhull.halfspace import Halfspace
from pyhull.halfspace import HalfspaceIntersection

def cdd_vertices(A, b, number_type="float"):
    b = b.reshape((-1, 1))
    inequality_list = np.hstack((b, -A)).tolist()
    inequality_matrix = cdd.Matrix(inequality_list, number_type=number_type)
    inequality_matrix.rep_type = cdd.RepType.INEQUALITY
    polyhedron = cdd.Polyhedron(inequality_matrix)
    generator_matrix = polyhedron.get_generators()
    generator_list = generator_matrix.__getitem__(slice(0, generator_matrix.row_size))
    vertices = np.vstack([np.array(map(float, generator_list[i][1:])) for i in range(0, len(generator_list)) if generator_list[i][0] == 1])
    return vertices

def qhull_vertices(A, b):
    b = b.reshape((-1, 1))
    interior_point = np.array([[-9.50611132], [-0.53511235], [-4.93859559], [-0.24137556]])
    halfspaces = []
    for i in range(0, A.shape[0]):
        halfspace = Halfspace(A[i,:].tolist(), (-b[i,0]).tolist())
        halfspaces.append(halfspace)
    polyhedron_qhull = HalfspaceIntersection(halfspaces, interior_point.flatten().tolist())
    vertices = np.vstack(polyhedron_qhull.vertices)
    return vertices
"""

# Comparing Python Results

cddlib in floating point mode finds only 15 vertices, while qhull finds 19. Fortunately, setting cddlib to use fractions instead of floats lets it find all 19 vertices as well. 

Note: we sort the rows of all of the results in order to get a consistent order for comparison later. 

In [17]:
cdd_verts_float = sortrows(py"cdd_vertices($A, $b)")

15×4 Array{Float64,2}:
 -9.92349   1.01009   -1.35573  -6.68365 
 -9.92257   1.00949   -1.36506  -6.67727 
 -9.91315   0.788092  -1.55392  -5.61185 
 -9.79254  -0.698839  -3.18811   1.39908 
 -9.65913   1.12537   -2.9788   -7.91233 
 -9.65658   1.12659   -2.99479  -7.9254  
 -9.63279   1.17765   -2.98819  -8.46959 
 -9.61456   0.781321  -3.46326  -6.3322  
 -9.53791   1.23063   -3.55394  -9.03416 
 -9.45377  -0.549825  -5.26325  -0.189102
 -9.43923  -0.538001  -5.33209  -0.315119
 -9.40882  -0.477651  -5.34292  -0.958323
 -9.32823  -0.43266   -5.82344  -1.43784 
 -9.31228  -0.42672   -5.9302   -1.50115 
 -9.30588  -0.398094  -5.94406  -1.68055 

In [6]:
cdd_verts_exact = sortrows(py"cdd_vertices($A, $b, 'fraction')")

19×4 Array{Float64,2}:
 -9.92349   1.01009   -1.35573  -6.68365 
 -9.92257   1.00949   -1.36506  -6.67727 
 -9.92171   1.01071   -1.37327  -6.69034 
 -9.91315   0.788092  -1.55392  -5.61185 
 -9.79254  -0.698839  -3.18811   1.39908 
 -9.71596   1.12037   -2.69997  -7.85909 
 -9.65913   1.12537   -2.9788   -7.91233 
 -9.65658   1.12659   -2.99479  -7.9254  
 -9.63279   1.17765   -2.98819  -8.46959 
 -9.61456   0.781321  -3.46326  -6.3322  
 -9.53791   1.23063   -3.55394  -9.03416 
 -9.52742  -0.557541  -4.89756  -0.106859
 -9.45377  -0.549825  -5.26325  -0.189102
 -9.43923  -0.538001  -5.33209  -0.315119
 -9.40882  -0.477651  -5.34292  -0.958323
 -9.32823  -0.43266   -5.82344  -1.43784 
 -9.31616  -0.42672   -5.91084  -1.50115 
 -9.31228  -0.42672   -5.9302   -1.50115 
 -9.30588  -0.398094  -5.94406  -1.68055 

In [7]:
qhull_verts = sortrows(py"qhull_vertices($A, $b)")

19×4 Array{Float64,2}:
 -9.92349   1.01009   -1.35573  -6.68365 
 -9.92257   1.00949   -1.36506  -6.67727 
 -9.92171   1.01071   -1.37327  -6.69034 
 -9.91315   0.788092  -1.55392  -5.61185 
 -9.79254  -0.698839  -3.18811   1.39908 
 -9.71596   1.12037   -2.69997  -7.85909 
 -9.65913   1.12537   -2.9788   -7.91233 
 -9.65658   1.12659   -2.99479  -7.9254  
 -9.63279   1.17765   -2.98819  -8.46959 
 -9.61456   0.781321  -3.46326  -6.3322  
 -9.53791   1.23063   -3.55394  -9.03416 
 -9.52742  -0.557541  -4.89756  -0.106859
 -9.45377  -0.549825  -5.26325  -0.189102
 -9.43923  -0.538001  -5.33209  -0.315119
 -9.40882  -0.477651  -5.34292  -0.958323
 -9.32823  -0.43266   -5.82344  -1.43784 
 -9.31616  -0.42672   -5.91084  -1.50115 
 -9.31228  -0.42672   -5.9302   -1.50115 
 -9.30588  -0.398094  -5.94406  -1.68055 

The cddlib+exact results agree with the qhull results:

In [8]:
all(isapprox.(cdd_verts_exact, qhull_verts))

true

We can also run the same computations in Julia, adding the ConvexHull.jl library as an additional choice:

In [9]:
function cdd_vertices(A, b, number_type=:float)
    h = SimpleHRepresentation(A, b)
    poly = polyhedron(h, CDDLib.CDDLibrary(number_type))
    Float64.(SimpleVRepresentation(vrep(poly)).V) 
end

function convex_hull_vertices(A, b, number_type=:float)
    h = SimpleHRepresentation(A, b)
    poly = polyhedron(h, ConvexHull.ConvexHullLib(number_type))
    Float64.(SimpleVRepresentation(vrep(poly)).V)
end

function qhull_vertices(A, b)
    h = SimpleHRepresentation(A, b)
    poly = polyhedron(h, QHull.QHullLib())
    Float64.(SimpleVRepresentation(vrep(poly)).V)
end

qhull_vertices (generic function with 1 method)

As expected, cddlib again only finds 15 vertices in floating point mode, but recovers all 19 in exact mode:

In [10]:
cdd_verts_julia_float = sortrows(cdd_vertices(A, b, :float))

15×4 Array{Float64,2}:
 -9.92349   1.01009   -1.35573  -6.68365 
 -9.92257   1.00949   -1.36506  -6.67727 
 -9.91315   0.788092  -1.55392  -5.61185 
 -9.79254  -0.698839  -3.18811   1.39908 
 -9.65913   1.12537   -2.9788   -7.91233 
 -9.65658   1.12659   -2.99479  -7.9254  
 -9.63279   1.17765   -2.98819  -8.46959 
 -9.61456   0.781321  -3.46326  -6.3322  
 -9.53791   1.23063   -3.55394  -9.03416 
 -9.45377  -0.549825  -5.26325  -0.189102
 -9.43923  -0.538001  -5.33209  -0.315119
 -9.40882  -0.477651  -5.34292  -0.958323
 -9.32823  -0.43266   -5.82344  -1.43784 
 -9.31228  -0.42672   -5.9302   -1.50115 
 -9.30588  -0.398094  -5.94406  -1.68055 

In [11]:
cdd_verts_julia_exact = sortrows(cdd_vertices(A, b, :exact))

19×4 Array{Float64,2}:
 -9.92349   1.01009   -1.35573  -6.68365 
 -9.92257   1.00949   -1.36506  -6.67727 
 -9.92171   1.01071   -1.37327  -6.69034 
 -9.91315   0.788092  -1.55392  -5.61185 
 -9.79254  -0.698839  -3.18811   1.39908 
 -9.71596   1.12037   -2.69997  -7.85909 
 -9.65913   1.12537   -2.9788   -7.91233 
 -9.65658   1.12659   -2.99479  -7.9254  
 -9.63279   1.17765   -2.98819  -8.46959 
 -9.61456   0.781321  -3.46326  -6.3322  
 -9.53791   1.23063   -3.55394  -9.03416 
 -9.52742  -0.557541  -4.89756  -0.106859
 -9.45377  -0.549825  -5.26325  -0.189102
 -9.43923  -0.538001  -5.33209  -0.315119
 -9.40882  -0.477651  -5.34292  -0.958323
 -9.32823  -0.43266   -5.82344  -1.43784 
 -9.31616  -0.42672   -5.91084  -1.50115 
 -9.31228  -0.42672   -5.9302   -1.50115 
 -9.30588  -0.398094  -5.94406  -1.68055 

In [12]:
all(isapprox.(cdd_verts_julia_exact, qhull_verts))

true

The pure-Julia ConvexHull.jl library is slower, but also finds all 19 vertices in exact mode:

In [13]:
convex_hull_verts_julia_exact = sortrows(convex_hull_vertices(A, b, :exact))

19×4 Array{Float64,2}:
 -9.92349   1.01009   -1.35573  -6.68365 
 -9.92257   1.00949   -1.36506  -6.67727 
 -9.92171   1.01071   -1.37327  -6.69034 
 -9.91315   0.788092  -1.55392  -5.61185 
 -9.79254  -0.698839  -3.18811   1.39908 
 -9.71596   1.12037   -2.69997  -7.85909 
 -9.65913   1.12537   -2.9788   -7.91233 
 -9.65658   1.12659   -2.99479  -7.9254  
 -9.63279   1.17765   -2.98819  -8.46959 
 -9.61456   0.781321  -3.46326  -6.3322  
 -9.53791   1.23063   -3.55394  -9.03416 
 -9.52742  -0.557541  -4.89756  -0.106859
 -9.45377  -0.549825  -5.26325  -0.189102
 -9.43923  -0.538001  -5.33209  -0.315119
 -9.40882  -0.477651  -5.34292  -0.958323
 -9.32823  -0.43266   -5.82344  -1.43784 
 -9.31616  -0.42672   -5.91084  -1.50115 
 -9.31228  -0.42672   -5.9302   -1.50115 
 -9.30588  -0.398094  -5.94406  -1.68055 

In [14]:
all(isapprox.(convex_hull_verts_julia_exact, qhull_verts))

true

...although it seems to have issues in floating point mode:

In [15]:
convex_hull_verts_julia_exact = sortrows(convex_hull_vertices(A, b, :float))

0×4 Array{Float64,2}

There are also issues with this particular interface to qhull. It appears to expect that the polyhedra contains the origin (which makes some sense, since we have to provide an interior point to qhull anyway):

In [16]:
qhull_verts_julia = sortrows(qhull_vertices(A, b))

LoadError: The origin should be in the interior of the polytope but the 1th inequality is not safisfied at the origin.